### Master table만들어보기

In [ ]:
# 앞서 한것에 이어서(4번) 마스터 테이블을 만들어보자

In [ ]:
%pyspark


artists = sqlContext.read.format("parquet").load("s3://spotify--artist/artists/dt=2020-02-13/artists.parquet")
artists.toDF("id", "name", "followers", "popularity", "url", "image_url")
artists.show()


import pandas as pd
import pymysql

host = ?
port = ?
username = ?
database = ?
password = ?

try:
    conn = pymysql.connect(host, user=username, passwd=password, db=database, port=port, use_unicode=True, charset='utf8')
    cursor = conn.cursor()
except:
    logging.error("could not connect to rds")
    sys.exit(1)
    
cursor.execute("SELECT * FROM artists")
colnames = [d[0] for d in cursor.description]
artists = [dict(zip(colnames, row)) for row in cursor.fetchall()]
artists = pd.DataFrame(artists)


artists = sqlContext.createDataFrame(artists)
artists.show()

top_tracks = sqlContext.read.format("parquet").load("s3://spotify--artist/top-tracks/dt=2020-02-11/top-tracks.parquet")
top_tracks = top_tracks.toDF("id", "artist_id", "name", "popularity", "external_url")
top_tracks = top_tracks.withColumnRenamed("id", "track_id").withColumnRenamed("name", "track_name")
top_tracks = top_tracks.select(top_tracks['track_id'][0].alias('track_id'), top_tracks["track_name"][0].alias('track_name'), top_tracks['artist_id'], top_tracks['popularity'][0].alias('track_popularity'))





joined = artists.join(top_tracks, top_tracks['artist_id'] == artists['id'])

features = sqlContext.read.format("parquet").load("s3://spotify--artist/audio-features/dt=2020-02-11/top-tracks.parquet")
features = features.toDF("danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "type", "id", "url", "track_href", "analysis_url", "duration_ms", "time_signature")


features = features.withColumnRenamed("id", "track_id")
master = joined.join(features, joined['track_id'] == features['track_id'])
master.show()

master.registerTempTable("master")

>>>
+--------------------+--------------------+---------+----------+--------------------+--------------------+
|                  id|                name|followers|popularity|                 url|           image_url|
+--------------------+--------------------+---------+----------+--------------------+--------------------+
|00FQb4jTyendYWaN8...|        Lana Del Rey|  9898011|        89|https://open.spot...|https://i.scdn.co...|
|01C9OoXDvCKkGcf73...|    Serge Gainsbourg|   297170|        63|https://open.spot...|https://i.scdn.co...|
|02rd0anEWfMtF7iMk...|       Reba McEntire|  1061502|        66|https://open.spot...|https://i.scdn.co...|
|02uYdhMhCgdB49hZl...|Eagles Of Death M...|   369692|        61|https://open.spot...|https://i.scdn.co...|
|03r4iKL2g2442PT9n...|        Beastie Boys|  1508984|        74|https://open.spot...|https://i.scdn.co...|
|03YhcM6fxypfwckPC...|      Wes Montgomery|   132611|        57|https://open.spot...|https://i.scdn.co...|
|04gDigrS5kc9YWfZH...|            Maroon 5| 22199288|        93|https://open.spot...|https://i.scdn.co...|
|04tBaW21jyUfeP5iq...|        Scott Walker|    76386|        52|https://open.spot...|https://i.scdn.co...|
|0543y7yrvny4Kymoa...|      Peter Frampton|   587596|        62|https://open.spot...|https://i.scdn.co...|
|05E3NBxNMdnrPtxF9...|        Lester Young|    86251|        53|https://open.spot...|https://i.scdn.co...|
|06HL4z0CvFAxyc27G...|        Taylor Swift| 26561227|        95|https://open.spot...|https://i.scdn.co...|
|06nevPmNVfWUXyZkc...|      Gregory Porter|   359073|        68|https://open.spot...|https://i.scdn.co...|
|06nsZ3qSOYZ2hPVIM...|           J.J. Cale|   370016|        66|https://open.spot...|https://i.scdn.co...|
|085pc2PYOi8bGKj0P...|           will.i.am|  2847810|        78|https://open.spot...|https://i.scdn.co...|
|08avsqaGIlK2x3i2C...|      Keith Richards|   159398|        48|https://open.spot...|https://i.scdn.co...|
|09C0xjtosNAIXP36w...|         Fats Domino|   268210|        61|https://open.spot...|https://i.scdn.co...|
|0BvkDsjIUla7X0k6C...|          Luke Bryan|  4965591|        80|https://open.spot...|https://i.scdn.co...|
|0bvRYuXRvd14RYEE7...|Linton Kwesi Johnson|    24391|        44|https://open.spot...|https://i.scdn.co...|
|0C0XlULifJtAgn6ZN...|         The Killers|  5140898|        82|https://open.spot...|https://i.scdn.co...|
|0cc6vw3VN8YlIcvr1...|         Mötley Crüe|  2377678|        77|https://open.spot...|https://i.scdn.co...|
+--------------------+--------------------+---------+----------+--------------------+--------------------+
only showing top 20 rows
+---------+--------------------+--------------------+--------------------+----------+--------------------+
|followers|                  id|           image_url|                name|popularity|                 url|
+---------+--------------------+--------------------+--------------------+----------+--------------------+
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|        Lana Del Rey|        89|https://open.spot...|
|   297170|01C9OoXDvCKkGcf73...|https://i.scdn.co...|    Serge Gainsbourg|        63|https://open.spot...|
|  1061502|02rd0anEWfMtF7iMk...|https://i.scdn.co...|       Reba McEntire|        66|https://open.spot...|
|   369692|02uYdhMhCgdB49hZl...|https://i.scdn.co...|Eagles Of Death M...|        61|https://open.spot...|
|  1508984|03r4iKL2g2442PT9n...|https://i.scdn.co...|        Beastie Boys|        74|https://open.spot...|
|   132611|03YhcM6fxypfwckPC...|https://i.scdn.co...|      Wes Montgomery|        57|https://open.spot...|
| 22199288|04gDigrS5kc9YWfZH...|https://i.scdn.co...|            Maroon 5|        93|https://open.spot...|
|    76386|04tBaW21jyUfeP5iq...|https://i.scdn.co...|        Scott Walker|        52|https://open.spot...|
|   587596|0543y7yrvny4Kymoa...|https://i.scdn.co...|      Peter Frampton|        62|https://open.spot...|
|    86251|05E3NBxNMdnrPtxF9...|https://i.scdn.co...|        Lester Young|        53|https://open.spot...|
| 26561227|06HL4z0CvFAxyc27G...|https://i.scdn.co...|        Taylor Swift|        95|https://open.spot...|
|   359073|06nevPmNVfWUXyZkc...|https://i.scdn.co...|      Gregory Porter|        68|https://open.spot...|
|   370016|06nsZ3qSOYZ2hPVIM...|https://i.scdn.co...|           J.J. Cale|        66|https://open.spot...|
|  2847810|085pc2PYOi8bGKj0P...|https://i.scdn.co...|           will.i.am|        78|https://open.spot...|
|   159398|08avsqaGIlK2x3i2C...|https://i.scdn.co...|      Keith Richards|        48|https://open.spot...|
|   268210|09C0xjtosNAIXP36w...|https://i.scdn.co...|         Fats Domino|        61|https://open.spot...|
|  4965591|0BvkDsjIUla7X0k6C...|https://i.scdn.co...|          Luke Bryan|        80|https://open.spot...|
|    24391|0bvRYuXRvd14RYEE7...|https://i.scdn.co...|Linton Kwesi Johnson|        44|https://open.spot...|
|  5140898|0C0XlULifJtAgn6ZN...|https://i.scdn.co...|         The Killers|        82|https://open.spot...|
|  2377678|0cc6vw3VN8YlIcvr1...|https://i.scdn.co...|         Mötley Crüe|        77|https://open.spot...|
+---------+--------------------+--------------------+--------------------+----------+--------------------+
only showing top 20 rows
+---------+--------------------+--------------------+------------+----------+--------------------+--------------------+--------------------+--------------------+----------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------+
|followers|                  id|           image_url|        name|popularity|                 url|            track_id|          track_name|           artist_id|track_popularity|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|          type|            track_id|                 uri|          track_href|        analysis_url|duration_ms|time_signature|
+---------+--------------------+--------------------+------------+----------+--------------------+--------------------+--------------------+--------------------+----------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------+
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|3hwQhakFwm9soLEBn...|        Venice Bitch|00FQb4jTyendYWaN8...|              71|       0.404| 0.298|  9| -12.986|   0|     0.0304|       0.912|           0.228|  0.0996|  0.315| 78.815|audio_features|3hwQhakFwm9soLEBn...|spotify:track:3hw...|https://api.spoti...|https://api.spoti...|     577199|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|6PUIzlqotEmPuBfjb...|Summertime Sadnes...|00FQb4jTyendYWaN8...|              72|       0.572|  0.81|  1|  -5.791|   0|     0.0558|      0.0157|         6.53E-6|    0.13|   0.11|126.052|audio_features|6PUIzlqotEmPuBfjb...|spotify:track:6PU...|https://api.spoti...|https://api.spoti...|     214912|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|6PUIzlqotEmPuBfjb...|Summertime Sadnes...|00FQb4jTyendYWaN8...|              72|       0.572|  0.81|  1|  -5.791|   0|     0.0558|      0.0157|         6.53E-6|    0.13|   0.11|126.052|audio_features|6PUIzlqotEmPuBfjb...|spotify:track:6PU...|https://api.spoti...|https://api.spoti...|     214912|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|6PUIzlqotEmPuBfjb...|Summertime Sadnes...|00FQb4jTyendYWaN8...|              72|       0.572|  0.81|  1|  -5.791|   0|     0.0558|      0.0157|         6.53E-6|    0.13|   0.11|126.052|audio_features|6PUIzlqotEmPuBfjb...|spotify:track:6PU...|https://api.spoti...|https://api.spoti...|     214912|             4|
|  9898011|00FQb4jTyendYWaN8...|https://i.scdn.co...|Lana Del Rey|        89|https://open.spot...|6PUIzlqotEmPuBfjb...|Summertime Sadnes...|00FQb4jTyendYWaN8...|              72|       0.572|  0.81|  1|  -5.791|   0|     0.0558|      0.0157|         6.53E-6|    0.13|   0.11|126.052|audio_features|6PUIzlqotEmPuBfjb...|spotify:track:6PU...|https://api.spoti...|https://api.spoti...|     214912|             4|
+---------+--------------------+--------------------+------------+----------+--------------------+--------------------+--------------------+--------------------+----------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------+
only showing top 20 rows

In [ ]:
%sql

SELECT name, popularity, AVG(abs(popularity - track_popularity)) as diff FROM master GROUP BY 1,2 ORDER BY 3 ASC LIMIT 20

>>>


In [ ]:
%sql

SELECT AVG(acousticness), AVG(liveness), AVG(speechiness), AVG(tempo) FROM master WHERE popularity > ${popularity=80}



<img src="./imgs/3_2.png" width="60%">

In [ ]:
# 안에 $표시 함으로 인해서, popularity를 zeppelin안에서 변경가능

In [ ]:
# spotify 에서 skewedness (acousticness)를보면 앞쪽으로 치우쳐짐. 이것을 근거로

In [ ]:
%sql

SELECT ROUND(acousticness, 1), COUNT(*) FROM master GROUP BY 1 ORDER BY 1 ASC

(bar chart 또한 그려보면 이렇게 distrubted 됬다는것을 알 수 있다.)